In [1]:
import numpy as np
from PIL import Image
import os
import cv2
import PIL

In [ ]:
os.mkdir('results/')

In [2]:
def messageToBinary(message):
    if type(message) == str:
        return ''.join([format(ord(i),'08b') for i in message])
    elif type(message) == bytes or type(message) == np.ndarray:
        return [format(i, '08b') for i in message]
    elif type(message)==int:
        return format(message,'08b')
    else:
        raise TypeError('Invalid Data Type')

In [3]:
def showImage(src,title,text=''):
    #print(src)
    image = cv2.imread(src)
    #print(image)
    height = image.shape[0]
    cv2.putText(image,text,(0,height//2), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)
    cv2.imshow(title,image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [4]:
def Encode(src, message, dest):
    showImage(src,'Original Image')
    img = Image.open(src).convert('RGB')
    width, height = img.size
    array = np.array(list(img.getdata()))

    if img.mode == 'RGB':
        n = 3
    elif img.mode == 'RGBA':
        n = 4

    total_pixels = array.size//n
    
    message += "&I$igm@"
    b_message = messageToBinary(message)
    req_pixels = len(b_message)

    if req_pixels > total_pixels:
        raise ValueError('Need A Large Image')

    else:
        index=0
        for p in range(total_pixels):
            for q in range(0, 3):
                if index < req_pixels:
                    array[p][q] = int(bin(array[p][q])[2:9] + b_message[index], 2)
                    index += 1

        array=array.reshape(height, width, n)
        enc_img = Image.fromarray(array.astype('uint8'), img.mode)
        enc_img.save(dest)
        print("Image Encoded Successfully")

In [5]:
def Decode(src):

    img = Image.open(src, 'r')
    array = np.array(list(img.getdata()))

    if img.mode == 'RGB':
        n = 3
    elif img.mode == 'RGBA':
        n = 4

    total_pixels = array.size//n

    hidden_bits = ""
    for p in range(total_pixels):
        for q in range(0, 3):
            hidden_bits += (bin(array[p][q])[2:][-1])

    hidden_bits = [hidden_bits[i:i+8] for i in range(0, len(hidden_bits), 8)]

    message = ""
    for i in range(len(hidden_bits)):
        if message[-7:] == "&I$igm@":
            break
        else:
            message += chr(int(hidden_bits[i], 2))
    if "&I$igm@" in message:
        print("Hidden Message:", message[:-7])
        showImage(src,title = "Decoded", text=message[:-7])
    else:
        print("No Hidden Message Found")

In [6]:
def main():
    Encode('test-images/pikachu.png',"Hello World","results/img1.png")
    #count += 1
main()


Image Encoded Successfully


In [7]:
#print(messageToBinary("ABC6$igm@"))
Decode('results/img1.png')

Hidden Message: Hello World
